In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from datetime import datetime
import pandas as pd
import json

In [2]:
url = 'https://mars.nasa.gov/news/'

In [3]:
response = requests.get(url)
response.status_code

200

In [4]:
html = response.text
html[0:50]

"<!DOCTYPE html>\n<html lang='en' xml:lang='en' xmln"

In [5]:
executable_path = {'executable_path': r"C:\Users\kylec\Desktop\chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [6]:
# soup = BeautifulSoup(html, "lxml")
html = browser.html
soup = BeautifulSoup(html,"html.parser")

In [7]:
# html = browser.html
# soup = BeautifulSoup(html, "lxml")

In [8]:
teasers = soup.find_all(class_="article_teaser_body")

len(teasers)

40

In [9]:
soup.find_all(class_="article_teaser_body")

[<div class="article_teaser_body">Now that the lander's robotic arm has helped the mole get underground, it will resume science activities that have been on hold.</div>,
 <div class="article_teaser_body">After more than a year in the "clay-bearing unit," Curiosity is making a mile-long journey around some deep sand so that it can explore higher up Mount Sharp.</div>,
 <div class="article_teaser_body">The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.</div>,
 <div class="article_teaser_body">The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. </div>,
 <div class="article_teaser_body">Learn more about the agency's next Red Planet mission during a live event on June 17.</div>,
 <div class="article_teaser_body">This new portrait of the Red Planet's neighbors was taken 

In [10]:
teasers = soup.find_all(class_="article_teaser_body")
newsTeasers = teasers[0].text.strip()
newsTeasers

"Now that the lander's robotic arm has helped the mole get underground, it will resume science activities that have been on hold."

In [11]:
factsUrl = 'https://space-facts.com/mars/'
browser.visit(factsUrl)
html = browser.html

soup = BeautifulSoup(html, "html.parser")

stats = soup.find('table', id='tablepress-p-mars')
eqDiameterLabel = stats.find_all('strong')[0].text
poDiameterLabel = stats.find_all('strong')[1].text
massLabel = stats.find_all('strong')[2].text
moonsLabel = stats.find_all('strong')[3].text
orbitDistLabel = stats.find_all('strong')[4].text
orbitPdLabel = stats.find_all('strong')[5].text
tempLabel = stats.find_all('strong')[6].text
firstRecLabel = stats.find_all('strong')[7].text
recByLabel = stats.find_all('strong')[8].text
eqDiameter = stats.find_all(class_='column-2')[0].text
poDiameter = stats.find_all(class_='column-2')[1].text
mass = stats.find_all(class_='column-2')[2].text
moons = stats.find_all(class_='column-2')[3].text
orbitdDist = stats.find_all(class_='column-2')[4].text
orbitdPd = stats.find_all(class_='column-2')[5].text
temp = stats.find_all(class_='column-2')[6].text
firstRec = stats.find_all(class_='column-2')[7].text
recBy = stats.find_all(class_='column-2')[8].text

In [12]:
def marsScraper():
    executable_path = {'executable_path': r"C:\Users\kylec\Desktop\chromedriver.exe"}
    browser = Browser('chrome', **executable_path, headless=True)
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    titles = soup.find_all(class_="content_title")
    
    firstTitle = "" 
    for title in titles:
        if title.a:
            firstTitle = title
            break
    
    newsTitle = firstTitle.a.text.strip()
    newsLink = "https://mars.nasa.gov" + firstTitle.a['href']
    
    teaser = soup.find_all(class_="article_teaser_body")

    
    newsTeasers = teaser[0].text.strip()
    
    imageUrl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(imageUrl)
    imageHtml = browser.html
    soups = BeautifulSoup(imageHtml, "lxml")
    
    images = soups.find_all(class_="carousel_item")
    imageURL = 'https://www.jpl.nasa.gov' + images[0]["style"].split(" ")[1].split("'")[1]
    
    twitterUrl = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(twitterUrl)
    twitterHtml = browser.html
    soupss = BeautifulSoup(twitterHtml, "lxml")
    
    allTweets = soupss.find_all("span")
    tweetText = ""
    for tweet in allTweets:
        if tweet.text:
            if "InSight sol" in tweet.text:
                tweetText = tweet.text
                break
    
    factsUrl = 'https://space-facts.com/mars/'
    browser.visit(factsUrl)
    factsHtml = browser.html
    dfs = pd.read_html(factsHtml)
    stats = dfs[0]
    stats.columns = ["Attribute", "Value"]
    
    #format and save
    data_html = stats.to_html(index=False)
    data_stats = json.loads(stats.to_json(orient="records"))

    browser.quit()
    
    retDict = {
        "newsTitle": newsTitle,
        "newsTeaser": newsTeasers,
        "newsLink": newsLink,
        "featureImageURL": imageURL,
        "marsStats": data_stats,
        "dateScraped": datetime.now()
    }
    
    return retDict

In [13]:
executable_path = {'executable_path': r"C:\Users\kylec\Desktop\chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
    
html = browser.html
soup = BeautifulSoup(html, "html.parser")
    
news = soup.find("li", class_='slide')

newsTitle = news.find('div',class_="content_title").get_text()

In [14]:
newsTitle

"NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause"

In [15]:
def marsScraper():
    executable_path = {'executable_path': r"C:\Users\kylec\Desktop\chromedriver.exe"}
    browser = Browser('chrome', **executable_path, headless=True)
    
    #Mars News
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    news = soup.find("li", class_='slide')

    newsTitle = news.find('div',class_="content_title").get_text()
    
    newsTeaser = soup.find("div",class_="article_teaser_body")
    
    #Picture
    imageUrl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(imageUrl)
    imageHtml = browser.html
    soups = BeautifulSoup(imageHtml, "lxml")
    
    images = soups.find_all(class_="carousel_item")
    imageURL = 'https://www.jpl.nasa.gov' + images[0]["style"].split(" ")[1].split("'")[1]
    
    #Twitter (I stole this from Alexander completely)
    twitterUrl = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(twitterUrl)
    twitterHtml = browser.html
    soupss = BeautifulSoup(twitterHtml, "lxml")
    
    allTweets = soupss.find_all("span")
    tweetText = ""
    for tweet in allTweets:
        if tweet.text:
            if "InSight sol" in tweet.text:
                tweetText = tweet.text
                break
    
    #Mars Facts
    factsUrl = 'https://space-facts.com/mars/'
    browser.visit(factsUrl)
    factsHtml = browser.html
    dfs = pd.read_html(factsHtml)
    stats = dfs[0]
    stats.columns = ["Attribute", "Value"]
    
    #format and save
    data_html = stats.to_html(index=False)
    data_stats = json.loads(stats.to_json(orient="records"))

    #Closing Browser
    browser.quit()
    
    #Returning Dictionary
    retDict = {
        "newsTitle": newsTitle,
        "newsTeaser": newsTeasers,
        "featureImageURL": imageURL,
        "marsStats": data_stats,
        "dateScraped": datetime.now()
    }
    
    return retDict

In [16]:
marsNews = marsScraper()
marsNews

{'newsTitle': "NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause",
 'newsTeaser': "Now that the lander's robotic arm has helped the mole get underground, it will resume science activities that have been on hold.",
 'featureImageURL': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA09320-1920x1200.jpg',
 'marsStats': [{'Attribute': 'Equatorial Diameter:', 'Value': '6,792 km'},
  {'Attribute': 'Polar Diameter:', 'Value': '6,752 km'},
  {'Attribute': 'Mass:', 'Value': '6.39 × 10^23 kg (0.11 Earths)'},
  {'Attribute': 'Moons:', 'Value': '2 (Phobos & Deimos)'},
  {'Attribute': 'Orbit Distance:', 'Value': '227,943,824 km (1.38 AU)'},
  {'Attribute': 'Orbit Period:', 'Value': '687 days (1.9 years)'},
  {'Attribute': 'Surface Temperature:', 'Value': '-87 to -5 °C'},
  {'Attribute': 'First Record:', 'Value': '2nd millennium BC'},
  {'Attribute': 'Recorded By:', 'Value': 'Egyptian astronomers'}],
 'dateScraped': datetime.datetime(2020, 7, 7, 18, 14, 31, 338761)}